In [1]:

!python -V

Python 3.11.11


In [2]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [3]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/03/26 14:23:54 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/kushalramaiya/Documents/GitHub/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1742979234730, experiment_id='1', last_update_time=1742979234730, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [11]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [13]:
df_train = read_dataframe('../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2021-02.parquet')

In [14]:
len(df_train), len(df_val)

(73908, 61921)

In [15]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [16]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [17]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [18]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758714882773319

In [19]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [22]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "../data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "../data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [21]:
import xgboost as xgb

In [23]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [24]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [25]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [26]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

In [27]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/kushalramaiya/miniconda3/envs/mlops-zoom/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [14:32:39] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.36698                           
[1]	validation-rmse:7.16821                           
[2]	validation-rmse:6.81895                           
[3]	validation-rmse:6.69015                           
[4]	validation-rmse:6.63527                           
[5]	validation-rmse:6.61467                           
[6]	validation-rmse:6.60266                           
[7]	validation-rmse:6.59577                           
[8]	validation-rmse:6.58836                           
[9]	validation-rmse:6.58510                           
[10]	validation-rmse:6.58060                          
[11]	validation-rmse:6.57833                          
[12]	validation-rmse:6.57292                          
[13]	validation-rmse:6.56905                          
[14]	validation-rmse:6.56319                          
[15]	validation-rmse:6.56169                          
[16]	validation-rmse:6.54928                          
[17]	validation-rmse:6.54611                          
[18]	valid

/Users/kushalramaiya/miniconda3/envs/mlops-zoom/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [14:32:52] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[2]	validation-rmse:6.75361                                                    
[3]	validation-rmse:6.70913                                                    
[4]	validation-rmse:6.69191                                                    
[5]	validation-rmse:6.68397                                                    
[6]	validation-rmse:6.68084                                                    
[7]	validation-rmse:6.67710                                                    
[8]	validation-rmse:6.67125                                                    
[9]	validation-rmse:6.66652                                                    
[10]	validation-rmse:6.65617                                                   
[11]	validation-rmse:6.64864                                                   
[12]	validation-rmse:6.64616                                                   
[13]	validation-rmse:6.64289                                                   
[14]	validation-rmse:6.63949            

...

[0]	validation-rmse:11.32498                                                   
[1]	validation-rmse:10.56290                                                   
[2]	validation-rmse:9.91256                                                    
[3]	validation-rmse:9.36117                                                    
[4]	validation-rmse:8.89528                                                    
[5]	validation-rmse:8.50343                                                    
[6]	validation-rmse:8.17479                                                    
[7]	validation-rmse:7.89986                                                    
[8]	validation-rmse:7.67152                                                    
[9]	validation-rmse:7.48057                                                    
[10]	validation-rmse:7.32258                                                   
[11]	validation-rmse:7.19220                                                   
[12]	validation-rmse:7.08471            

/Users/kushalramaiya/miniconda3/envs/mlops-zoom/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [15:01:52] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:11.73647                                                   
[1]	validation-rmse:11.29582                                                   
[2]	validation-rmse:10.88883                                                   
[3]	validation-rmse:10.51364                                                   
[4]	validation-rmse:10.16764                                                   
[5]	validation-rmse:9.84952                                                    
[6]	validation-rmse:9.55760                                                    
[7]	validation-rmse:9.28982                                                    
[8]	validation-rmse:9.04403                                                    
[9]	validation-rmse:8.81957                                                    
[10]	validation-rmse:8.61411                                                   
[11]	validation-rmse:8.42666                                                   
[12]	validation-rmse:8.25559            

/Users/kushalramaiya/miniconda3/envs/mlops-zoom/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [15:03:08] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[1]	validation-rmse:11.10334                                                   
[2]	validation-rmse:10.62770                                                   
[3]	validation-rmse:10.19937                                                   
[4]	validation-rmse:9.81342                                                    
[5]	validation-rmse:9.46677                                                    
[6]	validation-rmse:9.15688                                                    
[7]	validation-rmse:8.87982                                                    
[8]	validation-rmse:8.63329                                                    
[9]	validation-rmse:8.41476                                                    
[10]	validation-rmse:8.21777                                                   
[11]	validation-rmse:8.04402                                                   
[12]	validation-rmse:7.88918                                                   
[13]	validation-rmse:7.75214            

In [29]:
mlflow.xgboost.autolog(disable=True)

In [30]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09435998655149878,
        'max_depth': 39,
        'min_child_weight': 1.9487322107872531,
        'objective': 'reg:linear',
        'reg_alpha': 0.021436135009241304,
        'reg_lambda': 0.3646085937929378,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/Users/kushalramaiya/miniconda3/envs/mlops-zoom/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [15:07:17] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:11.45122
[1]	validation-rmse:10.78245
[2]	validation-rmse:10.19696
[3]	validation-rmse:9.68786
[4]	validation-rmse:9.24347
[5]	validation-rmse:8.86006
[6]	validation-rmse:8.52900
[7]	validation-rmse:8.24472
[8]	validation-rmse:8.00233
[9]	validation-rmse:7.79266
[10]	validation-rmse:7.61443
[11]	validation-rmse:7.46001
[12]	validation-rmse:7.33082
[13]	validation-rmse:7.21730
[14]	validation-rmse:7.12232
[15]	validation-rmse:7.04024
[16]	validation-rmse:6.96957
[17]	validation-rmse:6.90881
[18]	validation-rmse:6.85612
[19]	validation-rmse:6.81068
[20]	validation-rmse:6.77278
[21]	validation-rmse:6.74001
[22]	validation-rmse:6.71105
[23]	validation-rmse:6.68502
[24]	validation-rmse:6.66253
[25]	validation-rmse:6.64297
[26]	validation-rmse:6.62585
[27]	validation-rmse:6.60973
[28]	validation-rmse:6.59652
[29]	validation-rmse:6.58444
[30]	validation-rmse:6.57295
[31]	validation-rmse:6.56361
[32]	validation-rmse:6.55444
[33]	validation-rmse:6.54625
[34]	validation-rmse:

/Users/kushalramaiya/miniconda3/envs/mlops-zoom/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [15:08:10] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/03/26 15:08:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "../data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "../data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)